In [ ]:
from X_AutoMap_utils import *

# === Normalization and Dilation Function ===
def normalize_and_dilate(img):
    img = np.nan_to_num(img)
    norm = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    dilated = cv2.dilate(norm, np.ones((5, 5), np.uint8), iterations=3)
    return norm, dilated

# === Setup ===
notebook_dir = pathlib.Path().resolve()
watch_dir = notebook_dir / "input"
watch_dir.mkdir(exist_ok=True)

def find_unique_tiff_files(folder):
    return list({f for f in os.listdir(folder) if f.lower().endswith((".tiff", ".tif"))})

processed_jsons = set()
json_items = []
precomputed_blobs = {
    "red": {},
    "green": {},
    "blue": {}
}
print("Start")

while True:
    # Step 1: Look for new JSON files
    json_files = [f for f in os.listdir(watch_dir) if f.endswith(".json") and f not in processed_jsons]
    
    if json_files:
        for json_file in json_files:
            file_path = watch_dir / json_file
            print(f"\n--- Reading JSON: {json_file} ---")
            try:
                with open(file_path, "r") as f:
                    data = json.load(f)
                    if isinstance(data, dict):
                        json_items = [[key, value] for key, value in data.items()]
                        print("Stored JSON key-value pairs:")
                        for pair in json_items:
                            print(pair)
                    else:
                        json_items = [data]  # fallback for non-dict JSON
                        print(json_items)
            except Exception as e:
                print(f"Error reading {json_file}: {e}")
                continue
            
            processed_jsons.add(json_file)

            # Step 2: Wait for 3 unique TIFF files
            precomputed = {}
            print("\nWaiting for 3 unique .tiff files...")
            while True:
                tiff_files = find_unique_tiff_files(watch_dir)
                if len(tiff_files) >= 3:
                    first_three = sorted(tiff_files)[:3]
                    tiff1_path = watch_dir / first_three[0]
                    tiff2_path = watch_dir / first_three[1]
                    tiff3_path = watch_dir / first_three[2]

                    print("\n✅ Found 3 TIFF files:")
                    print(f"TIFF 1: {tiff1_path.name}")
                    print(f"TIFF 2: {tiff2_path.name}")
                    print(f"TIFF 3: {tiff3_path.name}")
                    try:
                        tiff1_img = tiff.imread(str(tiff1_path)).astype(np.float32)
                        tiff1_norm, tiff1_dilated = normalize_and_dilate(tiff1_img)

                        tiff2_img = tiff.imread(str(tiff2_path)).astype(np.float32)
                        tiff2_norm, tiff2_dilated = normalize_and_dilate(tiff2_img)

                        tiff3_img = tiff.imread(str(tiff3_path)).astype(np.float32)
                        tiff3_norm, tiff3_dilated = normalize_and_dilate(tiff3_img)
                        
                        b1 = detect_blobs(tiff1_dilated, tiff1_norm, json_items[0][1], json_items[1][1], "red", tiff1_path.name)
                        precomputed_blobs["red"][(json_items[0][1], json_items[1][1])] = b1

                        b2 = detect_blobs(tiff2_dilated, tiff2_norm, json_items[0][1], json_items[1][1], "blue", tiff2_path.name)
                        precomputed_blobs["blue"][(json_items[0][1], json_items[1][1])] = b2

                        b3 = detect_blobs(tiff3_dilated, tiff3_norm, json_items[0][1], json_items[1][1], "green", tiff3_path.name)
                        precomputed_blobs["green"][(json_items[0][1], json_items[1][1])] = b3

                        # print(precomputed_blobs)
                        
                        unions = find_union_blobs(precomputed_blobs, json_items[2][1], json_items[3][1], json_items[4][1], json_items[5][1])

                        print("\n✅ All TIFFs processed and stored in memory.")

                        # print(unions)
                        for idx, union in unions.items():
                            print(f"Union #{idx}")
                            print(f"  Pixel Center:        {union['center']}")
                            print(f"  Box Length (px):     {union['length']}")
                            print(f"  Box Area (px²):      {union['area']}")
                            print(f"  Real Center (µm):    ({union['real_center'][0]:.2f}, {union['real_center'][1]:.2f})")
                            print(f"  Real Size (µm):      ({union['real_size'][0]:.2f}, {union['real_size'][1]:.2f})")
                            print(f"  Real Area (µm²):     {union['real_area']:.2f}")
                            print(f"  Real Top Left (µm):  ({union['real_top_left'][0]:.2f}, {union['real_top_left'][1]:.2f})")
                            print(f"  Real Bottom Right:   ({union['real_bottom_right'][0]:.2f}, {union['real_bottom_right'][1]:.2f})")
                            print("-" * 50)

                        
                        # Example: send to processing function
                        # process_images(tiff1_dilated, tiff2_dilated, tiff3_dilated)

                    except Exception as e:
                        print(f"❌ Error processing TIFFs: {e}")
                        break

                    break
                else:
                    print(f"Currently found {len(tiff_files)} TIFF file(s). Waiting...")
                    time.sleep(5)
    else:
        # print("No new JSON files found.")
        time.sleep(1)


Start

--- Reading JSON: OpenCV_params.json ---
Stored JSON key-value pairs:
['min_threshold_intensity', 100]
['min_threshold_area', 200]
['microns_per_pixel_x', 1.1]
['microns_per_pixel_y', 1.2]
['true_origin_x', 10]
['true_origin_y', 20]
['number_of_scans', 3]
['debt_names', 'deb_fs']
['x_motor', '2pssx']
['x_start', 1]
['x_end', 1]
['y_motor', '2pssy']
['y_start', 2]
['y_end', 2]
['interval', 0]

Waiting for 3 unique .tiff files...

✅ Found 3 TIFF files:
TIFF 1: mosaic_200_Ca_merged.tiff
TIFF 2: mosaic_200_Fe_merged.tiff
TIFF 3: mosaic_200_S_merged.tiff

✅ All TIFFs processed and stored in memory.
Union #1
  Pixel Center:        (479, 753)
  Box Length (px):     24
  Box Area (px²):      576
  Real Center (µm):    (536.90, 923.60)
  Real Size (µm):      (26.40, 28.80)
  Real Area (µm²):     760.32
  Real Top Left (µm):  (523.70, 909.20)
  Real Bottom Right:   (550.10, 938.00)
--------------------------------------------------
Union #2
  Pixel Center:        (500, 611)
  Box Length (